# 0. Importación de librerias

In [1]:
# Librerías para el análisis de datos
import numpy as np  # Biblioteca para operaciones numéricas, especialmente útil para trabajar con matrices y arrays
import pandas as pd  # Biblioteca para manipulación y análisis de datos, especialmente para DataFrames
import statsmodels.api as sm  # Biblioteca para realizar modelos estadísticos, útil para regresiones y otros análisis estadísticos
from collections import Counter  # Contador de elementos en colecciones, útil para contar la frecuencia de elementos en una lista o array

# Librería para optimización
#from gurobipy import Model, GRB, quicksum  # Gurobi es una biblioteca de optimización matemática. 
# Model permite definir el modelo de optimización,
# GRB contiene constantes (como tipos de variables y sentido de optimización),
# quicksum permite realizar sumas de forma rápida y eficiente en Gurobi.

# Librerías operacionales
import os  # Biblioteca para interactuar con el sistema operativo (ej. manejo de archivos y rutas)
from datetime import datetime  # Módulo para trabajar con fechas y tiempos, útil para capturar fechas actuales o manipular datos de tiempo


# <h2>1. Carga de datos desde excel a dataframes de pandas</h2>


En primer lugar se cargan los datos iniciales desde los csv que nos proveen. (Esto puede tardar un ratito)

In [2]:
url_tratamiento = 'https://drive.usercontent.google.com/download?id=1KTRwYGaWoQQnZwbk8VpdxKULMaOReoF5&authuser=0&confirm=t&uuid=4e4d7983-c5a5-412d-9f5b-b9bda1068b73&at=AENtkXZSxkDr84kWrDlz6ANq4ov2%3A1730951312566'

df_tratamiento = pd.read_csv(url_tratamiento)

In [3]:
file_id = '173bRZQG7NWfdpHJ-o4-NA3ieH1-Fhyok'
url_informacion_de_clientes = f'https://drive.google.com/uc?id={file_id}'

df_informacion_de_clientes = pd.read_csv(url_informacion_de_clientes)

In [4]:
url_simulaciones_clientes = 'https://drive.usercontent.google.com/download?id=1IXyKwtKFLCUsAV1MtNqktiwKaHzV2D5A&authuser=0&confirm=t&uuid=edd22376-238f-4c1f-b603-728b07bafd7f&at=AENtkXaEURpV52p_BdWxyisvjhSQ%3A1730951026384'

df_simulaciones_clientes = pd.read_csv(url_simulaciones_clientes)

In [5]:
file_id = '1Z4jMzZeD2q-4-ioSgyimppAdLZzqDkt3'
url_ventas = f'https://drive.google.com/uc?id={file_id}'

df_ventas = pd.read_csv(url_ventas)

## <h3>1.1 Carga de 'Informacion_Clientes.csv'</h3>

En primer lugar se cargará la información de los clientes. Esto incluye las siguientes características de los clientes:

* **unnamed**: algo como uid
* **Rut**: identificador de Chile (supongo que por privacidad va desde 0 a max de observaciones)
* **Género**: Masculino o femenino
* **Categoría_Digital**: Si el cliente es digital o no
* **Elasticidad_Precios**: Baja, media o alta
* **Nacionalidad**: Chileno o extranjero
* **Propensión**: Número entre 0 y 1 que idica que tan propenso a cursar un credito es el cliente
* **Probabilidad_No_Pago**: Número entre 0 y 1 que indica la probabilidad de que el cliente no pague la deuda
* **Edad**: Numero entero de edad en años
* **Renta**: Renta promedio de los últimos 12 meses
* **Oferta_Consumo**: Monto máximo que puede cursar un cliente dado sus antecedentes crediticios y situación socioeconómica. 
* **Deuda_CMF**: Deuda que tiene el cliente en otros bancos. Efectivamente es deuda pendiente, pero de créditos otorgados por la competencia.
* **Tiempo_como_cliente**: Número de tiempo(no sé en que medida está) que el cliente lleva en el banco


Se elimina el tiempo como cliente ya que no aporta información

In [6]:
df_informacion_de_clientes.drop(columns=['Tiempo_como_cliente'], inplace=True)

## <h3>1.2 Carga de 'Simulaciones_Clientes.csv'</h3>



En segundo lugar se cargaran las simulaciones hechas por los clientes en la página del banco. Esto incluye las siguientes características de las simulaciones:
* **unnamed**: Supongo que es el número de simulacion registrada, un tipo de identificador de la simulación
* **fecha**: yyyy-mm-dd fecha de la simulación
* **rut**: identificador de Chile del cliente que hizo la simulacion
* **monto_simulado**: monto prestado al cliente
* **plazo_simulado**: plazo en **meses** del crédito
* **tasa_simulado**: costo para el cliente del credito

## <h3>1.3 Carga de 'Tratamiento.csv'</h3>

En tercer lugar se cargara el tratamiento que ha tenido el banco con el cliente, es decir, cómo se han contactado con él. Esto incluye las siguientes características:

* **unnamed**: Número de tratamiento registrado
* **fecha**: yyyy-mm-dd
* **rut**: Identificador de Chile del cliente con el que se tiene el tipo de trato
* **n_correos**: Cantidad de correos que se enviaron en el mes que sale la fecha. Es decir, si sele fecha '2024-03-01', correspondería a los correos enviados en marzo de 2024.
* **asg_ejec**: Si el cliente tiene un ejecutivo asignado

## <h3>1.4 Carga de 'Ventas.csv'</h3>

Por último se cargaran las ventas que ha tenido el banco con el cliente. Esto incluye las siguientes características:

* **unnamed**: Índice sin significado
* **fecha**: yyyy-mm-dd -> fecha en la que se concretó la venta
* **rut**: identificador de Chile del cliente al que se le concretó la venta

# <h2>2. Joints de datos<h2>

In [7]:
# Unir los DataFrames 'df_informacion_de_clientes' y 'df_simulaciones_clientes' en base a la columna 'rut'
# El método 'how="left"' asegura que todos los registros de 'df_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_simulaciones_clientes'.
df_simulaciones_e_informacion_de_clientes = pd.merge(
    df_informacion_de_clientes, 
    df_simulaciones_clientes, 
    on='rut', 
    how='left'
)

# Crear una nueva columna 'simulo' que indica si el cliente tiene un 'Monto_Simulado' o no
# El método 'notna()' devuelve True para valores no nulos y False para nulos.
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes['simulo'] = df_simulaciones_e_informacion_de_clientes['Monto_Simulado'].notna().astype(int)

# Eliminar columnas innecesarias 'Unnamed: 0_x' y 'Unnamed: 0_y' que podrían haber surgido durante la carga o manipulación de datos
df_simulaciones_e_informacion_de_clientes.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)


In [8]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes' y 'df_ventas' en base a las columnas 'rut' y 'fecha'
# El método 'how="left"' asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes' se conserven,
# incluso si no tienen coincidencia en 'df_ventas'.
df_simulaciones_e_informacion_de_clientes_ventas = pd.merge( 
    df_simulaciones_e_informacion_de_clientes, 
    df_ventas, 
    on=['rut', 'fecha'], 
    how='left'
)

# Crear una nueva columna 'venta' que indica si existe una venta asociada al cliente y la fecha específica
# El método 'notna()' verifica si hay un valor no nulo en la columna 'Unnamed: 0' (que indica presencia de una venta)
# Luego, 'astype(int)' convierte estos valores booleanos en enteros (1 para True, 0 para False).
df_simulaciones_e_informacion_de_clientes_ventas['venta'] = df_simulaciones_e_informacion_de_clientes_ventas['Unnamed: 0'].notna().astype(int)


In [9]:
# Unir los DataFrames 'df_simulaciones_e_informacion_de_clientes_ventas' y 'df_tratamiento' en base a las columnas 'rut' y 'fecha'
# La unión se realiza con 'how="left"' para conservar todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas'
# incluso si no tienen coincidencia en 'df_tratamiento'.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento = pd.merge( 
    df_simulaciones_e_informacion_de_clientes_ventas, 
    df_tratamiento, 
    on=['rut', 'fecha'], 
    how='left'
)

# Renombrar la columna 'Unnamed: 0_x' a 'venta' para tener una denominación clara y comprensible.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento.rename(columns={'Unnamed: 0_x': 'venta'}, inplace=True)

# Actualizar la columna 'venta' para que indique si el registro de venta no es nulo (1 si hay venta, 0 si no hay)
# Se utiliza 'notna()' para verificar la presencia de un valor y 'astype(int)' para convertir el resultado booleano a entero.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'] = df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['venta'].notna().astype(int)

# Crear una nueva columna 'mes' que extrae el mes y año de la columna 'fecha'
# Primero se convierte 'fecha' al formato datetime, luego 'dt.to_period('M')' obtiene el periodo del mes/año.
df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['mes'] = pd.to_datetime(df_simulaciones_e_informacion_de_clientes_ventas_tratamiento['fecha']).dt.to_period('M')


# 3. CLUSTERING POR POLITICAS

### Seteo de cluster. Aquí se definen las variables y sus cortes. La idea es que el algoritmo de RL haga sus acciones en esta sección

Aquí se definen que variables se utilizarán para crear el cluster

In [10]:
df_informacion_de_clientes_procesados_cluster_definitivo = df_informacion_de_clientes[['rut', 'Categoria_Digital', 'Edad', 'Genero', 'Renta', 'Propension', 'Probabilidad_No_Pago']].copy()


Aqui se definen en que partes y en cuantas partes se particionarán las variables escogidas anteriormente

In [11]:
# Crear una copia del DataFrame 'df_informacion_de_clientes_procesados_cluster_definitivo' para trabajar sin modificar el original
df = df_informacion_de_clientes_procesados_cluster_definitivo.copy()

# Clasificar la columna 'Propension' en tres categorías usando cuantiles, lo que permite dividir los datos en terciles
# Las etiquetas indican si la propensión es baja, media o alta.
df['Categoria_Propenso'] = pd.qcut(df['Propension'], 3, labels=['Propension baja', 'Propension media', 'Propension alta'])

# Clasificar la columna 'Probabilidad_No_Pago' en cinco categorías, asignando etiquetas según los valores de probabilidad
# Cada categoría representa el nivel de confiabilidad en el pago: desde 'Muy buen pagador' hasta 'Muy mal pagador'.
df['Categoria_Probabilidad_No_Pago'] = pd.cut(df['Probabilidad_No_Pago'], 
                                              bins=[-float('inf'), 0.0011, 0.00149, 0.005, 0.006, float('inf')],
                                              labels=['Muy buen pagador', 'Buen pagador', 'Pagador neutro', 'Mal pagador', 'Muy mal pagador'])

# Clasificar la columna 'Edad' en tres categorías: 'Joven', 'Adulto' y 'Adulto Mayor'
# Cada categoría se define en función de rangos de edad especificados en 'bins'.
df['Categoria_Edad'] = pd.cut(df['Edad'], 
                              bins=[-float('inf'), 35, 60, float('inf')],
                              labels=['Joven', 'Adulto', 'Adulto Mayor'])

# Crear un DataFrame único de 'rut' y 'Renta' eliminando duplicados, para calcular percentiles de renta
df_unicos_renta = df[['rut', 'Renta']].drop_duplicates()

# Calcular el percentil de renta para cada cliente usando 'qcut' para dividir en 100 grupos
# Añadimos 1 para que los percentiles inicien en 1 en lugar de 0.
df_unicos_renta['Percentil_Renta'] = pd.qcut(df_unicos_renta['Renta'], 100, labels=False) + 1

# Clasificar la columna 'Percentil_Renta' en tres categorías: 'Renta Baja', 'Renta Media' y 'Renta Alta'
# Los rangos de percentil especificados en 'bins' definen estas categorías.
df_unicos_renta['Categoria_Renta'] = pd.cut(df_unicos_renta['Percentil_Renta'], 
                                            bins=[-float('inf'), 30, 80, float('inf')],
                                            labels=['Renta Baja', 'Renta Media', 'Renta Alta'])

# Incorporar la categoría de renta al DataFrame principal 'df' realizando una unión ('merge') en base a la columna 'rut'
df = df.merge(df_unicos_renta[['rut', 'Categoria_Renta']], on='rut', how='left')

# Mostrar el DataFrame resultante con las nuevas columnas creadas
df


,rut,Categoria_Digital,Edad,Genero,Renta,Propension,Probabilidad_No_Pago,Categoria_Propenso,Categoria_Probabilidad_No_Pago,Categoria_Edad,Categoria_Renta
0,1,Cliente no Digital,30.0,Masculino,6.258183e+05,0.997340,0.028445,Propension alta,Muy mal pagador,Joven,Renta Media
1,2,Cliente no Digital,41.0,Femenino,3.172616e+05,0.291601,0.014320,Propension baja,Muy mal pagador,Adulto,Renta Baja
2,3,Cliente no Digital,38.0,Femenino,1.240551e+07,0.685085,0.002156,Propension alta,Pagador neutro,Adulto,Renta Alta
3,4,Cliente no Digital,57.0,Masculino,5.441466e+05,0.914672,0.034418,Propension alta,Muy mal pagador,Adulto,Renta Baja
4,5,Cliente Digital,26.0,Masculino,1.870225e+05,0.425077,0.014978,Propension media,Muy mal pagador,Joven,Renta Baja
...,...,...,...,...,...,...,...,...,...,...,...
543646,543647,Cliente Digital,29.0,Femenino,1.176598e+05,0.144657,0.037291,Propension baja,Muy mal pagador,Joven,Renta Baja
543647,543648,Cliente no Digital,31.0,Masculino,1.558612e+06,0.740170,0.035877,Propension alta,Muy mal pagador,Joven,Renta Media
543648,543649,Cliente no Digital,49.0,Masculino,9.449508e+05,0.255285,0.023306,Propension baja,Muy mal pagador,Adulto,Renta Media
543649,543650,Cliente no Digital,40.0,Femenino,1.039964e+06,0.709086,0.015121,Propension alta,Muy mal pagador,Adulto,Renta Media


In [12]:
# Concatenar las variables especificadas en una nueva columna 'categoria_clusterizacion'
# La columna resultante combinará varias categorías en una descripción detallada del perfil del cliente.
# Convertimos cada columna a tipo string para asegurarnos de que los datos sean compatibles para la concatenación.

df['categoria_clusterizacion'] = (
    df['Categoria_Digital'].astype(str) + ' ' +              # Categoría de digitalización del cliente
    df['Categoria_Edad'].astype(str) + ' de genero ' +       # Categoría de edad, seguida de la palabra "de genero"
    df['Genero'].astype(str) + ' con ' +                     # Género del cliente
    df['Categoria_Propenso'].astype(str) + ' con una ' +     # Categoría de propensión, seguida de "con una"
    df['Categoria_Renta'].astype(str)                        # Categoría de renta
)


In [13]:
# Asignar un número único a cada entrada distinta en la columna 'categoria_clusterizacion'
# Se convierte la columna a tipo 'category', lo cual facilita la asignación de códigos numéricos únicos.
# 'cat.codes' asigna un código numérico único para cada valor único de 'categoria_clusterizacion'.
df['categoria_clusterizacion_numerica'] = df['categoria_clusterizacion'].astype('category').cat.codes


In [14]:
# Crear una copia del DataFrame con solo las columnas 'rut', 'categoria_clusterizacion' y 'categoria_clusterizacion_numerica'
# Esta copia se almacena en el nuevo DataFrame 'asignacion_clusters', el cual contendrá únicamente la identificación del cliente (rut),
# la descripción del perfil ('categoria_clusterizacion') y el código numérico asignado a cada perfil ('categoria_clusterizacion_numerica').
asignacion_clusters = df[['rut', 'categoria_clusterizacion', 'categoria_clusterizacion_numerica']].copy()


## 4. Estimacion de curvas de elasticidad por cluster

In [15]:
# Realizar una unión entre 'df_simulaciones_e_informacion_de_clientes_ventas' y 'asignacion_clusters' usando la columna 'rut' como clave
# Esta unión ('merge') se realiza con 'how="left"', lo que asegura que todos los registros de 'df_simulaciones_e_informacion_de_clientes_ventas' 
# se conserven, incluyendo aquellos sin coincidencia en 'asignacion_clusters'.
# La finalidad es agregar la información de clusterización (categoría y código numérico) al DataFrame de simulaciones y ventas.
df_estimar_elasticidad = pd.merge(df_simulaciones_e_informacion_de_clientes_ventas, asignacion_clusters, on='rut', how='left')


### Este código realiza un análisis de elasticidad de ingresos en función de clusters de clientes. Primero, agrupa los datos por clusters definidos a través de variables de segmentación y filtra solo los datos relevantes para cada cluster. Luego, para cada cluster, se ajusta un modelo de regresión logística para predecir la probabilidad de aceptación de una simulación de crédito en función de la tasa de interés. A partir de este modelo, se crea una cuadrícula de tasas para estimar la probabilidad de aceptación y calcular el revenue potencial de cada simulación, teniendo en cuenta el monto medio simulado, el plazo medio simulado y la probabilidad media de no pago del cluster. Posteriormente, se determina la tasa que maximiza el revenue esperado y se calcula el número esperado de créditos aceptados, junto con el número de clientes únicos en cada cluster. Finalmente, los resultados se agregan tanto en listas globales como en un nuevo DataFrame, y luego se integran en el DataFrame original df_estimar_elasticidad, lo que permite analizar el revenue esperado total y otros indicadores clave en cada cluster.

In [16]:
# Inicializar listas para almacenar resultados globales de revenue, clientes, créditos y simulaciones
lista_revenue = []
lista_clientes = []
lista_creditos = []
lista_simulaciones = []

cluster_results = []  # Lista para almacenar resultados específicos de cada cluster

# Obtener los números únicos de cada cluster
cluster_numbers = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()

# Iterar sobre cada cluster identificado por 'categoria_clusterizacion_numerica'
for cluster_num in cluster_numbers:
    # Filtrar los datos correspondientes al cluster actual
    df_cluster = df_estimar_elasticidad[df_estimar_elasticidad['categoria_clusterizacion_numerica'] == cluster_num]
    
    # Asegurarse de que existen datos para ambos casos: venta == 1 y venta == 0
    if df_cluster.empty or df_cluster['venta'].isnull().all():
        continue  # Saltar este cluster si no cumple con la condición
    
    # Remover filas donde 'venta' o 'Tasa_Simulado' son nulos o infinitos
    df_cluster = df_cluster.replace([np.inf, -np.inf], np.nan)
    df_cluster = df_cluster.dropna(subset=['venta', 'Tasa_Simulado', 'Plazo_Simulado', 'Monto_Simulado', 'Probabilidad_No_Pago'])
    
    # Saltar el cluster si no hay suficientes puntos de datos
    if df_cluster.shape[0] < 10:
        continue
    
    # Extraer las variables 'venta' (como variable dependiente) y 'Tasa_Simulado' (como predictor)
    y = df_cluster['venta']
    X = df_cluster[['Tasa_Simulado']]
    
    # Añadir un término constante para el intercepto
    X = sm.add_constant(X)
    
    # Remover filas con valores NaN o Inf en X o y
    is_finite = np.isfinite(X).all(1) & np.isfinite(y)
    X = X[is_finite]
    y = y[is_finite]
    
    # Asegurarse de que después de remover NaN/Inf, todavía hay suficientes datos
    if len(y) < 10:
        continue
    
    # Ajustar el modelo de regresión logística
    logit_model = sm.Logit(y, X)
    try:
        result = logit_model.fit(disp=0)
    except:
        continue  # Saltar el cluster si el modelo no converge
    
    # Crear una cuadrícula de valores de 'Tasa_Simulado' para predicciones
    tasa_min = df_cluster['Tasa_Simulado'].min()
    tasa_max = df_cluster['Tasa_Simulado'].max()
    tasas_grid = np.linspace(tasa_min, tasa_max, 1000)
    
    # Predecir la probabilidad de aceptación usando el modelo ajustado
    X_grid = sm.add_constant(tasas_grid)
    acceptance_probability = result.predict(X_grid)
    
    # Asegurar que las probabilidades están en el rango [0, 1]
    acceptance_probability = np.clip(acceptance_probability, 0, 1)
    
    # Calcular valores medios necesarios para el cálculo de revenue
    n = df_cluster['Plazo_Simulado'].mean()
    vp = df_cluster['Monto_Simulado'].mean()
    pnp = df_cluster['Probabilidad_No_Pago'].mean()
    data = {
        'Plazo_Simulado_medio': n, 
        'Monto_Simulado_medio': vp, 
        'Probabilidad_No_Pago_media': pnp
    }
    
    # Calcular el revenue potencial
    i = tasas_grid / 100  # Convertir a decimal
    one_plus_i_pow_n = np.power(1 + i, n)
    annuity_factor = (i * one_plus_i_pow_n) / (one_plus_i_pow_n - 1)
    revenue = (n * vp * annuity_factor) - vp
    potential_revenue = revenue * (1 - pnp)
    
    # Calcular el promedio de simulaciones por fecha
    df_cluster_simulaciones_1 = df_cluster[df_cluster['simulo'] == 1]
    num_dates = df_cluster_simulaciones_1['fecha'].nunique()
    total_simulaciones = df_cluster_simulaciones_1['simulo'].sum()
    simulaciones_medias = total_simulaciones / num_dates if num_dates else 0
    
    # Saltar el cluster si no hay simulaciones
    if simulaciones_medias == 0:
        continue
    
    # Calcular el revenue esperado
    expected_revenue = acceptance_probability * potential_revenue * simulaciones_medias
    
    # Encontrar la tasa que maximiza el revenue esperado
    idx_max = np.argmax(expected_revenue)
    max_price = tasas_grid[idx_max]
    max_expected_revenue = expected_revenue[idx_max]
    
    # Probabilidad de aceptación en la tasa óptima
    prob_aceptacion_optima = acceptance_probability[idx_max]
    
    # Número esperado de créditos aceptados
    num_creditos_aceptados = round(prob_aceptacion_optima * simulaciones_medias)
    
    # Número de clientes únicos en el cluster
    num_clients = df_cluster['rut'].nunique()
    
    # Imprimir resultados para cada cluster
    print(f'Cluster {cluster_num}:')
    print(f'- Precio Máx. Revenue Esperado = {max_price:.2f}%')
    print(f'- Revenue Esperado Máximo = {max_expected_revenue:,.2f}')
    print(f'- Número de clientes en el cluster = {num_clients}')
    print(f'- Número de simulaciones en el cluster = {simulaciones_medias:.2f}')
    print(f'- Probabilidad de aceptación en el precio óptimo = {prob_aceptacion_optima:.4f}')
    print(f'- Número esperado de créditos aceptados = {num_creditos_aceptados}')
    print(f'- Monto medio simulado = {data["Monto_Simulado_medio"]:,.2f}')
    print(f'- Plazo medio simulado = {data["Plazo_Simulado_medio"]:,.2f}')
    print(f'- Probabilidad de no pago media = {data["Probabilidad_No_Pago_media"]:.4f}\n')
    
    # Agregar resultados a las listas globales
    lista_clientes.append(num_clients)
    lista_revenue.append(max_expected_revenue)
    lista_creditos.append(num_creditos_aceptados)
    lista_simulaciones.append(simulaciones_medias)
    
    # Almacenar resultados por cluster en cluster_results
    cluster_results.append({
        'categoria_clusterizacion_numerica': cluster_num,
        'tasa_optima': max_price,
        'probabilidad_aceptacion_optima': prob_aceptacion_optima,
        'revenue_esperado_maximo': max_expected_revenue,
        'numero_clientes': num_clients,
        'numero_simulaciones_medias': simulaciones_medias,
        'numero_creditos_esperados': num_creditos_aceptados,
        'monto_medio_simulado': data["Monto_Simulado_medio"],
        'plazo_medio_simulado': data["Plazo_Simulado_medio"],
        'probabilidad_no_pago_media': data["Probabilidad_No_Pago_media"]
    })

# Imprimir resultados globales
total_revenue = sum(lista_revenue)
total_clientes = sum(lista_clientes)
total_simulaciones = sum(lista_simulaciones)
total_creditos = sum(lista_creditos)

print(f"El revenue total esperado es: {total_revenue:,.2f} con un total de {total_clientes} clientes, "
      f"{total_simulaciones:,.2f} simulaciones, y {total_creditos} créditos.")

# Crear un DataFrame a partir de cluster_results
df_cluster_results = pd.DataFrame(cluster_results)

# Incorporar los resultados por cluster de 'df_cluster_results' a 'df_estimar_elasticidad'
df_estimar_elasticidad = df_estimar_elasticidad.merge(
    df_cluster_results[['categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']],
    on='categoria_clusterizacion_numerica', 
    how='left'
)


Cluster 101:
- Precio Máx. Revenue Esperado = 1.26%
- Revenue Esperado Máximo = 446,098,007.80
- Número de clientes en el cluster = 6435
- Número de simulaciones en el cluster = 1817.58
- Probabilidad de aceptación en el precio óptimo = 0.4990
- Número esperado de créditos aceptados = 907
- Monto medio simulado = 2,666,668.32
- Plazo medio simulado = 27.47
- Probabilidad de no pago media = 0.0276

Cluster 76:
- Precio Máx. Revenue Esperado = 1.22%
- Revenue Esperado Máximo = 65,495,606.51
- Número de clientes en el cluster = 5907
- Número de simulaciones en el cluster = 912.80
- Probabilidad de aceptación en el precio óptimo = 0.5523
- Número esperado de créditos aceptados = 504
- Monto medio simulado = 712,403.67
- Plazo medio simulado = 27.65
- Probabilidad de no pago media = 0.0092

Cluster 72:
- Precio Máx. Revenue Esperado = 1.08%
- Revenue Esperado Máximo = 3,990,650,405.36
- Número de clientes en el cluster = 5431
- Número de simulaciones en el cluster = 1628.33
- Probabilidad d

# 5. Estimacion de respuesta a tratamiento por cluster

In [17]:
# Unir los DataFrames 'df_tratamiento' y 'df_simulaciones_clientes' usando las columnas 'rut' y 'fecha' como claves
# La unión se realiza con 'how="left"', lo cual asegura que todos los registros de 'df_tratamiento' se conserven,
# incluyendo aquellos sin coincidencia en 'df_simulaciones_clientes'.
# Esta operación permite combinar la información de tratamiento con los datos de simulaciones de clientes.
df_simulaciones_info = pd.merge(df_tratamiento, df_simulaciones_clientes, on=['rut', 'fecha'], how='left')


In [18]:
# Usar operaciones de cadenas vectorizadas para crear la columna 'Tratamiento'
# Esta columna concatenará información sobre el ejecutivo asignado y el número de correos enviados.
# Se convierte 'asg_ejec' a string para poder concatenar, y 'n_correos' se convierte primero a entero y luego a string.
# El formato final es: "Ejecutivo=<valor_asg_ejec>, Correos=<valor_n_correos>"
df_simulaciones_info['Tratamiento'] = (
    'Ejecutivo=' + df_simulaciones_info['asg_ejec'].astype(str) +
    ', Correos=' + df_simulaciones_info['n_correos'].astype(int).astype(str)
)


In [19]:
# Extraer el mes y año de la columna 'fecha' y crear una nueva columna 'mes' en formato de periodo mensual
# Se convierte 'fecha' al formato datetime y luego se usa 'dt.to_period('M')' para obtener el mes/año.
df_simulaciones_info['mes'] = pd.to_datetime(df_simulaciones_info['fecha']).dt.to_period('M')

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# lo cual es útil para verificar datos o analizar un cliente en particular.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05
...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05


In [20]:
# Crear una nueva columna 'simulo' para indicar si el cliente tiene un registro de simulación
# La columna 'Unnamed: 0_y' se utiliza para verificar si hay un valor no nulo, lo que implica que hay una simulación.
# 'notna()' devuelve True para valores no nulos y False para valores nulos; luego, 'astype(int)' convierte estos valores a 1 (True) o 0 (False).
df_simulaciones_info['simulo'] = df_simulaciones_info['Unnamed: 0_y'].notna().astype(int)

# Filtrar y mostrar las filas donde 'rut' es igual a 1
# Este filtro permite observar los registros específicos del cliente con 'rut' igual a 1, 
# útil para verificar si la columna 'simulo' refleja correctamente la presencia de simulaciones para este cliente.
df_simulaciones_info[df_simulaciones_info['rut'] == 1]


,Unnamed: 0_x,fecha,rut,n_correos,asg_ejec,Unnamed: 0_y,Monto_Simulado,Plazo_Simulado,Tasa_Simulado,Tratamiento,mes,simulo
0,0,2019-01-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-01,0
543651,0,2019-02-01,1,4,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=4",2019-02,0
1087302,0,2019-03-01,1,1,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=1",2019-03,0
1630953,0,2019-04-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2019-04,0
2174604,0,2019-05-01,1,0,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=0",2019-05,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33162711,0,2024-02-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-02,0
33706362,0,2024-03-01,1,3,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=3",2024-03,0
34250013,0,2024-04-01,1,2,0,NaN,NaN,NaN,NaN,"Ejecutivo=0, Correos=2",2024-04,0
34793664,0,2024-05-01,1,2,1,7864785.0,267268.0,31.0,2.027869,"Ejecutivo=1, Correos=2",2024-05,1


In [21]:
# Crear una copia del DataFrame 'df_estimar_elasticidad' con solo las columnas especificadas
# 'df1' contiene las columnas 'rut', 'categoria_clusterizacion_numerica', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
# Esta copia es útil para trabajar con los datos de elasticidad y clusterización sin modificar el DataFrame original.
df1 = df_estimar_elasticidad[['rut', 'categoria_clusterizacion_numerica', 'tasa_optima', 'probabilidad_aceptacion_optima']].copy()

# Crear una copia del DataFrame 'df_simulaciones_info' con solo las columnas especificadas
# 'df2' contiene las columnas 'rut', 'mes', 'Tratamiento' y 'simulo'.
# Esta copia es útil para trabajar con los datos de tratamiento y simulación en un conjunto de datos reducido.
df2 = df_simulaciones_info[['rut', 'mes', 'Tratamiento', 'simulo']].copy()


In [22]:
# Paso 1: Preparación de datos y mapeo de clusters
# Eliminar duplicados en 'df1' para tener un valor único de 'categoria_clusterizacion_numerica' por cada 'rut'.
df1_unique = df1.drop_duplicates(subset='rut')

# Crear un mapeo de 'rut' a 'categoria_clusterizacion_numerica' para asociar cada cliente a su cluster numérico.
rut_cluster_map = df1_unique.set_index('rut')['categoria_clusterizacion_numerica']

# Mapear la categoría de cluster a cada 'rut' en 'df2' usando el mapeo creado
df2['categoria_clusterizacion_numerica'] = df2['rut'].map(rut_cluster_map)

# Eliminar filas donde 'categoria_clusterizacion_numerica' es nulo, es decir, aquellos 'rut' sin mapeo de cluster.
df2 = df2.dropna(subset=['categoria_clusterizacion_numerica'])

# Conversión de tipos de datos
# Convertir 'categoria_clusterizacion_numerica' a entero para garantizar un tipo de dato consistente.
df2['categoria_clusterizacion_numerica'] = df2['categoria_clusterizacion_numerica'].astype(int)

# Convertir 'simulo' a numérico, reemplazando valores nulos por 0 y asegurando que sea un tipo de dato entero.
df2['simulo'] = pd.to_numeric(df2['simulo'], errors='coerce').fillna(0).astype(int)

# Convertir 'Tratamiento' a tipo de categoría para optimizar espacio y realizar operaciones categóricas.
df2['Tratamiento'] = df2['Tratamiento'].astype('category')

# Paso 2: Calcular el caso total (entradas por tratamiento sin importar el valor de 'simulo')
# Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número total de registros en cada combinación.
total_entries_per_cluster_treatment = df2.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_total')

# Paso 3: Calcular el caso favorable (entradas por tratamiento cuando 'simulo' == 1)
# Filtrar filas donde 'simulo' es 1 (clientes que realizaron una simulación)
df_simulations = df2[df2['simulo'] == 1]

# Agrupar por 'categoria_clusterizacion_numerica' y 'Tratamiento' para contar el número de registros favorables (simulaciones).
favorable_entries_per_cluster_treatment = df_simulations.groupby(['categoria_clusterizacion_numerica', 'Tratamiento']).size().reset_index(name='caso_favorable')

# Paso 4: Calcular la probabilidad de simulación como caso favorable / caso total
# Realizar un merge entre 'total_entries_per_cluster_treatment' y 'favorable_entries_per_cluster_treatment' en las columnas de cluster y tratamiento.
df_probabilities = total_entries_per_cluster_treatment.merge(
    favorable_entries_per_cluster_treatment,
    on=['categoria_clusterizacion_numerica', 'Tratamiento'],
    how='left'
)

# Llenar valores nulos en 'caso_favorable' con 0, asegurando que solo las columnas numéricas estén afectadas.
df_probabilities['caso_favorable'] = df_probabilities['caso_favorable'].fillna(0).astype(int)

# Asegurar que 'caso_total' sea de tipo entero para evitar inconsistencias en los conteos.
df_probabilities['caso_total'] = df_probabilities['caso_total'].astype(int)

# Calcular la probabilidad de simulación como el cociente entre 'caso_favorable' y 'caso_total'.
df_probabilities['probabilidad_simular'] = df_probabilities['caso_favorable'] / df_probabilities['caso_total']

# Organizar las columnas del DataFrame resultante para facilitar su análisis.
df_probabilities = df_probabilities[[
    'categoria_clusterizacion_numerica',
    'Tratamiento',
    'probabilidad_simular',
    'caso_favorable',
    'caso_total'
]]

# Mostrar el DataFrame resultante con la probabilidad de simulación calculada para cada combinación de cluster y tratamiento.
df_probabilities


,categoria_clusterizacion_numerica,Tratamiento,probabilidad_simular,caso_favorable,caso_total
0,0,"Ejecutivo=0, Correos=0",0.238938,27,113
1,0,"Ejecutivo=0, Correos=1",0.201613,25,124
2,0,"Ejecutivo=0, Correos=2",0.222222,18,81
3,0,"Ejecutivo=0, Correos=3",0.247104,64,259
4,0,"Ejecutivo=0, Correos=4",0.250000,96,384
...,...,...,...,...,...
859,107,"Ejecutivo=0, Correos=3",0.152868,10595,69308
860,107,"Ejecutivo=0, Correos=4",0.152917,17032,111381
861,107,"Ejecutivo=1, Correos=0",0.342740,6024,17576
862,107,"Ejecutivo=1, Correos=1",0.350587,6685,19068


In [23]:
# Visualizar el DataFrame 'df_estimar_elasticidad' para revisar su contenido antes de realizar cálculos adicionales
df_estimar_elasticidad

# Calcular el valor promedio de 'Monto_Simulado' para cada 'categoria_clusterizacion_numerica'
# Usamos 'groupby' para agrupar por 'categoria_clusterizacion_numerica' y 'transform("mean")' para calcular el promedio.
# Luego, 'transform' asigna este valor promedio a cada fila dentro de su grupo, creando una columna 'Monto_Simulado_mean' con estos promedios.
df_estimar_elasticidad['Monto_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Monto_Simulado'].transform('mean')

# Calcular el valor promedio de 'Plazo_Simulado' para cada 'categoria_clusterizacion_numerica'
# Similar al cálculo anterior, 'groupby' agrupa los datos por 'categoria_clusterizacion_numerica', y 'transform("mean")' calcula el promedio.
# Se asigna el promedio resultante a cada fila dentro del grupo en la nueva columna 'Plazo_Simulado_mean'.
df_estimar_elasticidad['Plazo_Simulado_mean'] = df_estimar_elasticidad.groupby('categoria_clusterizacion_numerica')['Plazo_Simulado'].transform('mean')


In [24]:
# Seleccionar solo las columnas necesarias del DataFrame 'df_estimar_elasticidad' para reducir su tamaño
# 'df_estimar_elasticidad_small' contiene las columnas esenciales para el análisis:
# 'categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago',
# 'Monto_Simulado_mean', y 'Plazo_Simulado_mean'.
df_estimar_elasticidad_small = df_estimar_elasticidad[['categoria_clusterizacion_numerica', 'rut', 'tasa_optima', 'probabilidad_aceptacion_optima', 'Probabilidad_No_Pago', 'Monto_Simulado_mean', 'Plazo_Simulado_mean']]

# Seleccionar solo las columnas necesarias del DataFrame 'df_probabilities' para reducir su tamaño
# 'df_probabilities_small' contiene las columnas 'categoria_clusterizacion_numerica', 'probabilidad_simular', y 'Tratamiento'.
df_probabilities_small = df_probabilities[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']]

# Realizar un merge entre 'df_estimar_elasticidad_small' y 'df_probabilities_small' usando 'categoria_clusterizacion_numerica' como clave
# Esta unión ('how="left"') mantiene todas las filas de 'df_estimar_elasticidad_small' y añade la información de 'df_probabilities_small'
# cuando hay coincidencias en 'categoria_clusterizacion_numerica'. El resultado se guarda en 'df_asignacion_de_tratamientos'.
df_asignacion_de_tratamientos = pd.merge(df_estimar_elasticidad_small, df_probabilities_small, on='categoria_clusterizacion_numerica', how='left')


In [25]:
# Crear un nombre de carpeta con una marca de tiempo actual
# 'strftime' genera la fecha y hora actual en el formato "YYYYMMDD_HHMMSS".
# Esto se usa para crear una carpeta única 'folder_name' donde se guardarán los archivos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"cluster_data_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si ya existe.

# Guardar información de los clusters
# Seleccionar las columnas relevantes sobre cada cluster desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# El DataFrame 'df_cluster_info' contiene datos únicos de cada cluster como el monto y plazo medio simulado, la probabilidad de aceptación óptima y la tasa óptima.
df_cluster_info = df_estimar_elasticidad_small[['categoria_clusterizacion_numerica', 'Monto_Simulado_mean', 'Plazo_Simulado_mean', 'probabilidad_aceptacion_optima', 'tasa_optima']].drop_duplicates()

# Guardar las probabilidades y tratamiento
# Seleccionar columnas relevantes de 'df_probabilities_small' para almacenar la probabilidad de simulación y tratamiento asignado para cada cluster.
# 'df_probabilities_treatment' contiene esta información única por cada combinación de cluster y tratamiento.
df_probabilities_treatment = df_probabilities_small[['categoria_clusterizacion_numerica', 'probabilidad_simular', 'Tratamiento']].drop_duplicates()

# Guardar información del RUT
# Seleccionar columnas clave sobre cada cliente ('rut') desde 'df_estimar_elasticidad_small' y eliminar duplicados.
# 'df_rut_info' contiene el 'rut', la categoría de cluster y la probabilidad de no pago para cada cliente, sin registros duplicados.
df_rut_info = df_estimar_elasticidad_small[['rut', 'categoria_clusterizacion_numerica', 'Probabilidad_No_Pago']].drop_duplicates()


Ahora se implementa el aprendizaje reforzado. En esta primera iteración se evalúa la mejora de clusters considerando el mismo n para cada uno. Además, como son solamente 10 episodios igual son pocas iteraciones para evaluar la recompensa obtenida.

In [26]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from datetime import datetime

# Suposición de que ya tienes tu DataFrame df_estimar_elasticidad
# df_estimar_elasticidad = pd.DataFrame(...)  # Aquí va tu DataFrame real

# Selección de las características para la creación de los clusters
features = ['Propension', 'Probabilidad_No_Pago', 'Edad', 'Renta']

# Definir el número máximo de clusters por categoría
MAX_CLUSTERS = 6

# Parámetros de aprendizaje reforzado (estos pueden ajustarse)
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 0.1  # Tasa de exploración

# Inicializar la Q-table (tabla de valores de acción)
# En este caso, la tabla contiene un valor para cada combinación de categoría y número de clusters posibles.
q_values = {}

# Inicializar el espacio de estados
categorias = df_estimar_elasticidad['categoria_clusterizacion_numerica'].unique()

# Crear una función para generar los nuevos clusters con KMeans
def generar_nuevos_clusters(cluster):
    """
    Esta función toma el índice del cluster actual y genera un número de nuevos clusters según
    la acción del agente en el aprendizaje reforzado.
    El número máximo de clusters es 6.
    """
    # Obtener el número de clusters a generar basado en la acción (número de clusters)
    num_clusters = np.random.randint(2, MAX_CLUSTERS + 1)  # Aseguramos que haya al menos 2 clusters y como máximo 6.
    
    # Aplicar KMeans para crear los nuevos clusters
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    df_estimar_elasticidad['nuevo_cluster'] = kmeans.fit_predict(df_estimar_elasticidad[features])
    
    # Imprimir el resultado de los nuevos clusters por categoría
    print(f"Cluster: {cluster} - Número de nuevos clusters: {num_clusters}")
    print(df_estimar_elasticidad.groupby('nuevo_cluster')[features].mean())
    print("\n")

    return df_estimar_elasticidad['nuevo_cluster'].values

# Función para calcular la recompensa (aquí es un ejemplo de cómo podría ser)
def calcular_recompensa(cluster, nuevos_clusters):
    """
    Esta función calcula la recompensa basada en la tasa de simulación, la probabilidad de aceptación y
    el revenue potencial. En este ejemplo, se usan algunas métricas simuladas.
    """
    # Calcular métricas de recompensa ficticias (aquí puedes usar las que consideres más relevantes)
    tasa_simulacion = np.random.rand()  # Simula un valor para la tasa de simulación
    probabilidad_aceptacion = np.random.rand()  # Simula un valor para la probabilidad de aceptación
    revenue_potencial = np.random.rand() * 1000  # Simula un valor para el revenue potencial
    
    recompensa = tasa_simulacion + probabilidad_aceptacion + revenue_potencial
    return recompensa

# Función para elegir la acción con una política epsilon-greedy
def elegir_accion(cluster):
    """
    Esta función implementa la política epsilon-greedy para elegir la acción (número de clusters).
    Si se selecciona una acción aleatoria (exploración), se elige un número aleatorio de clusters.
    Si no, se elige la acción con la mayor recompensa estimada (explotación).
    """
    if np.random.rand() < epsilon:  # Exploración
        return np.random.randint(2, MAX_CLUSTERS + 1)
    else:  # Explotación
        # Si no hay valores previos, inicializamos con un valor aleatorio
        if cluster not in q_values:
            q_values[cluster] = np.zeros(MAX_CLUSTERS)
        return np.argmax(q_values[cluster]) + 2  # Aseguramos que el número mínimo de clusters sea 2

# Función principal de aprendizaje reforzado
def aprendizaje_reforzado():
    """
    Esta función realiza el proceso de aprendizaje reforzado a lo largo de varios episodios.
    Cada episodio simula un ciclo de elección de acción, generación de nuevos clusters y cálculo de recompensa.
    """
    for episodio in range(1, 11):  # Cambia el número de episodios según sea necesario
        print(f"\nEpisodio {episodio}:\n")
        
        # Selección de un cluster aleatorio de las categorías existentes
        cluster = np.random.choice(categorias)
        
        # Elegir la acción (número de nuevos clusters)
        accion = elegir_accion(cluster)
        
        # Generar los nuevos clusters
        nuevos_clusters = generar_nuevos_clusters(cluster)
        
        # Calcular la recompensa para esta acción
        recompensa = calcular_recompensa(cluster, nuevos_clusters)
        
        # Actualizar la Q-table con el nuevo valor Q
        if cluster not in q_values:
            q_values[cluster] = np.zeros(MAX_CLUSTERS)
        q_values[cluster][accion - 2] = (1 - alpha) * q_values[cluster][accion - 2] + alpha * recompensa
        
        # Imprimir el resultado de los clusters generados
        print(f"Cluster {cluster} - Acción: {accion} clusters generados.")
        print(f"Recompensa obtenida: {recompensa}\n")
        
        # Aquí puedes agregar más lógica de seguimiento o almacenamiento de resultados

# Ejecutar el proceso de aprendizaje reforzado
aprendizaje_reforzado()



Episodio 1:



c:\Users\itzae\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


Cluster: 76 - Número de nuevos clusters: 3
               Propension  Probabilidad_No_Pago       Edad         Renta
nuevo_cluster                                                           
0                0.567976              0.018198  39.461768  1.405875e+06
1                0.562190              0.001438  51.958427  3.580543e+07
2                0.561218              0.008610  45.045315  9.095555e+06


Cluster 76 - Acción: 2 clusters generados.
Recompensa obtenida: 631.3612880962181


Episodio 2:



...

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 177, in where
KeyboardInterrupt: 


Cluster: 71 - Número de nuevos clusters: 3
               Propension  Probabilidad_No_Pago       Edad         Renta
nuevo_cluster                                                           
0                0.567976              0.018198  39.461768  1.405875e+06
1                0.562190              0.001438  51.958427  3.580543e+07
2                0.561218              0.008610  45.045315  9.095555e+06


Cluster 71 - Acción: 2 clusters generados.
Recompensa obtenida: 639.9310578500947


Episodio 10:



c:\Users\itzae\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster: 37 - Número de nuevos clusters: 4
               Propension  Probabilidad_No_Pago       Edad         Renta
nuevo_cluster                                                           
0                0.568205              0.018706  39.074654  1.116064e+06
1                0.561366              0.004798  47.547575  1.674522e+07
2                0.560650              0.000765  52.978582  4.636568e+07
3                0.563533              0.011408  43.871936  5.805776e+06


Cluster 37 - Acción: 2 clusters generados.
Recompensa obtenida: 39.77671953341857



La idea de este es que directamente mejore los clusters asignados originalmente, solo modificando la frontera en cada uno para ver si mejora o no. Falta incorporar las funciones reales pq ahora la recompensa es generada de forma random.

In [32]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

# Definir el DataFrame base (copia del original) y las características seleccionadas
df = df_informacion_de_clientes_procesados_cluster_definitivo.copy()
features = ['Propension', 'Probabilidad_No_Pago', 'Edad', 'Renta']

# Inicialización de las fronteras para cada categoría
fronteras = {
    'Propension': np.linspace(df['Propension'].min(), df['Propension'].max(), 4),  # 3 divisiones uniformes
    'Edad': [35, 60],  # Bajo 35, entre 35 y 60, mayor de 60
    'Renta': np.linspace(df['Renta'].min(), df['Renta'].max(), 4),  # 3 divisiones uniformes
    'Probabilidad_No_Pago': [0.0011, 0.00149, 0.005, 0.006]  # 5 divisiones con fronteras específicas
}

# Parámetros de aprendizaje
alpha = 0.1
q_values = {}
epsilon = 0.1

# Función para crear categorías dinámicamente basadas en fronteras
def crear_categorias(df, fronteras):
    df['Categoria_Propenso'] = pd.cut(df['Propension'], bins=fronteras['Propension'], labels=['Baja', 'Media', 'Alta'])
    df['Categoria_Probabilidad_No_Pago'] = pd.cut(df['Probabilidad_No_Pago'], bins=[-float('inf')] + fronteras['Probabilidad_No_Pago'] + [float('inf')], 
                                                  labels=['Muy buen pagador', 'Buen pagador', 'Neutro', 'Mal pagador', 'Muy mal pagador'])
    df['Categoria_Edad'] = pd.cut(df['Edad'], bins=[-float('inf')] + fronteras['Edad'] + [float('inf')], 
                                  labels=['Joven', 'Adulto', 'Adulto Mayor'])
    df['Categoria_Renta'] = pd.cut(df['Renta'], bins=fronteras['Renta'], labels=['Baja', 'Media', 'Alta'])
    df['Subgrupo'] = df['Categoria_Propenso'].astype(str) + '-' + df['Categoria_Probabilidad_No_Pago'].astype(str) + '-' + df['Categoria_Edad'].astype(str) + '-' + df['Categoria_Renta'].astype(str)
    return df

# Crear las categorías iniciales
df = crear_categorias(df, fronteras)

# Inicializar los subgrupos y Q-values
subgrupos = df['Subgrupo'].unique()

# Función para elegir la acción (ajustar las fronteras) usando epsilon-greedy
def elegir_accion(subgrupo):
    if np.random.rand() < epsilon:  # Exploración
        return np.random.choice(list(fronteras.keys()))
    else:  # Explotación
        if subgrupo not in q_values:
            q_values[subgrupo] = {k: 0 for k in fronteras}
        return max(q_values[subgrupo], key=q_values[subgrupo].get)

# Ajuste de las fronteras en función de la recompensa
def ajustar_frontera(categoria):
    if categoria == 'Edad':
        fronteras['Edad'] = [f + np.random.choice([-1, 1]) for f in fronteras['Edad']]
    elif categoria == 'Propension':
        limites = fronteras['Propension']
        cambio = np.random.uniform(-0.01, 0.01, size=3)  # Cambios pequeños en fronteras de propensión
        fronteras['Propension'] = np.sort(limites + cambio)
    elif categoria == 'Renta':
        limites = fronteras['Renta']
        cambio = np.random.uniform(-0.05, 0.05, size=3)  # Cambios pequeños en fronteras de renta
        fronteras['Renta'] = np.sort(limites + cambio)
    elif categoria == 'Probabilidad_No_Pago':
        limites = fronteras['Probabilidad_No_Pago']
        cambio = np.random.uniform(-0.0001, 0.0001, size=4)  # Ajustes leves en las fronteras de probabilidad de no pago
        fronteras['Probabilidad_No_Pago'] = np.sort(limites + cambio)

# Generar clusters y calcular recompensa
def generar_clusters(subgrupo, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    mask = df['Subgrupo'] == subgrupo
    df.loc[mask, 'nuevo_cluster'] = kmeans.fit_predict(df.loc[mask, features])
    
    # Calcular recompensa ficticia en función de la dispersión de propensión y probabilidad de no pago
    #calcular estas metricas de acuerdo a las funciones de mas arriba
    prob_simulacion = np.random.rand()  # Simular un valor para la probabilidad de simulación
    prob_aceptacion = np.random.rand()  # Simular un valor para la probabilidad de aceptación
    revenue_potencial = np.random.rand() * 1000  # Simular un valor para el revenue potencial
    recompensa = 1000*prob_simulacion + 1000*prob_aceptacion + revenue_potencial
    
    print(f"Subgrupo {subgrupo} - Clusters generados: {num_clusters}")
    print(df.loc[mask].groupby('nuevo_cluster')[features].mean())
    print(f"Recompensa obtenida: {recompensa}\n")
    return recompensa

# Función de aprendizaje reforzado
def aprendizaje_reforzado():
    for episodio in range(1, 11):  # Número de episodios
        print(f"\nEpisodio {episodio}:\n")
        
        # Seleccionar subgrupo aleatoriamente y decidir ajuste de frontera o número de clusters
        subgrupo = np.random.choice(subgrupos)
        categoria_a_ajustar = elegir_accion(subgrupo)
        
        # Ajustar la frontera de la categoría seleccionada y actualizar las categorías en el DataFrame
        ajustar_frontera(categoria_a_ajustar)
        crear_categorias(df, fronteras)
        
        # Generar clusters y calcular recompensa
        recompensa = generar_clusters(subgrupo, num_clusters=3)
        
        # Actualizar la Q-table
        if subgrupo not in q_values:
            q_values[subgrupo] = {k: 0 for k in fronteras}
        q_values[subgrupo][categoria_a_ajustar] = (1 - alpha) * q_values[subgrupo][categoria_a_ajustar] + alpha * recompensa

# Ejecutar el aprendizaje reforzado
aprendizaje_reforzado()



Episodio 1:



ValueError: operands could not be broadcast together with shapes (4,) (3,) 

# 6. Modelo de asignacion

## Modelo de asignacion que itera por cliente

In [ ]:
# # Complete Optimized Code: Data Processing and Optimization

# import pandas as pd
# import os
# import numpy as np
# from gurobipy import Model, GRB, quicksum

# # -------------------------------
# # Data Processing and Preprocessing
# # -------------------------------

# # Define the base folder and mappings
# base_folder = "cluster_data_20241106_020021/"
# tratamiento_map = {
#     "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
#     "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
#     "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
#     "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
# }

# # File paths
# probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
# rut_info_file = os.path.join(base_folder, "rut_info.csv")
# cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# # Parameters
# costosms = 100
# capacidad_ejecutivos = 205000

# # Step 1: Load and map 'tratamiento_id' in the probabilities data
# print("Loading and processing probabilities data...")
# df_probabilities = df_probabilities_treatment
# df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# # Step 2: Create 'tratamientos' list and merge with rut_info
# print("Merging probabilities with rut_info...")
# df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
# grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

# df_rut_info = df_rut_info
# df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')

# # Step 3: Merge rut_info with cluster_info
# print("Merging rut_info with cluster_info...")
# df_cluster_info = df_cluster_info
# df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# # Step 4: Calculate 'RC'
# print("Calculating RC...")
# df_rut_info['tasa_optima'] /= 100
# df_rut_info['RC'] = (
#     (df_rut_info['Plazo_Simulado_mean'] * df_rut_info['Monto_Simulado_mean'] * df_rut_info['tasa_optima'] *
#      ((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean'])) /
#     (((1 + df_rut_info['tasa_optima']) ** df_rut_info['Plazo_Simulado_mean']) - 1)
# ) - df_rut_info['Monto_Simulado_mean']

# # -------------------------------
# # Data Preparation for Optimization
# # -------------------------------

# # Convert 'tratamientos' to numpy array to improve indexing performance
# profits = np.array([
#     [
#         (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][t][0]) - 
#         (row['tratamientos'][t][1] * costosms)
#         for t in range(8)
#     ]
#     for _, row in df_rut_info.iterrows()
# ])

# # Initialize model
# model = Model("Maximizar_Ganancias")
# model.ModelSense = GRB.MAXIMIZE

# # Create decision variables and set objective
# n_clients, n_treatments = profits.shape
# variables = {}

# for i in range(n_clients):
#     variables[i] = {}
#     for t in range(n_treatments):
#         if profits[i, t] > 0:
#             variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

# model.setObjective(
#     quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
# )

# # Constraint: Each client receives exactly one treatment
# for i in variables:
#     model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# # Capacity constraint for executives
# model.addConstr(
#     quicksum(variables[i][t] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
#     name="CapacityConstraint"
# )

# # Cluster consistency
# clusters = df_rut_info.groupby("categoria_clusterizacion_numerica").indices
# for cluster_id, indices_cluster in clusters.items():
#     indices_list = list(indices_cluster)
#     leader_index = indices_list[0]
#     for t in variables[leader_index]:
#         leader_var = variables[leader_index][t]
#         for i in indices_list[1:]:
#             if t in variables[i]:
#                 model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

In [ ]:

# # Initialize model
# model = Model("Maximizar_Ganancias")
# model.ModelSense = GRB.MAXIMIZE

# # Create decision variables and set objective
# n_clients, n_treatments = profits.shape
# variables = {}

# for i in range(n_clients):
#     variables[i] = {}
#     for t in range(n_treatments):
#         if profits[i, t] > 0:
#             variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

# model.setObjective(
#     quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
# )

# # Constraint: Each client receives exactly one treatment
# for i in variables:
#     model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# # Capacity constraint for executives
# model.addConstr(
#     quicksum(variables[i][t] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
#     name="CapacityConstraint"
# )

# # Cluster consistency
# clusters = df_rut_info.groupby("categoria_clusterizacion_numerica").indices
# for cluster_id, indices_cluster in clusters.items():
#     indices_list = list(indices_cluster)
#     leader_index = indices_list[0]
#     for t in variables[leader_index]:
#         leader_var = variables[leader_index][t]
#         for i in indices_list[1:]:
#             if t in variables[i]:
#                 model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

# # Optimize the model
# model.optimize()

# # Check if the optimization was successful
# if model.Status == GRB.OPTIMAL:
#     # -------------------------------
#     # Extracting and Displaying Results
#     # -------------------------------

#     print("Extracting results...")

#     # Assign treatments per cluster based on the optimization results
#     resultados_por_cluster = {}
#     for cluster_id, indices_cluster in clusters.items():
#         leader_index = list(indices_cluster)[0]
#         for t in variables[leader_index]:
#             if variables[leader_index][t].X > 0.5:
#                 resultados_por_cluster[cluster_id] = t
#                 break

#     # Calculate total profits
#     ganancias_totales = model.ObjVal

#     # Display results
#     print("\nTratamientos asignados por cluster:")
#     for cluster_id, tratamiento in resultados_por_cluster.items():
#         print(f"Cluster {cluster_id}: Tratamiento {tratamiento + 1}")

#     print(f"\nGanancias totales: {ganancias_totales:.2f}")

#     # -------------------------------
#     # Calculating Executive Usage
#     # -------------------------------
    
#     # Count the number of executives used for treatments 5, 6, and 7
#     executives_used = sum(
#         1 for i in variables for t in variables[i] if t in [5, 6, 7] and variables[i][t].X > 0.5
#     )
#     executives_remaining = capacidad_ejecutivos - executives_used

#     # Display executive usage summary
#     print(f"\nExecutives used: {executives_used}")
#     print(f"Executives remaining: {executives_remaining}")

# else:
#     print("Optimization did not reach an optimal solution.")
# print("Optimization complete.")

In [ ]:
# from collections import Counter

# for cluster_id, tratamiento in resultados_por_cluster.items():
#     # Count the number of times each treatment is assigned
#     treatment_counts = Counter(resultados_por_cluster.values())

# # Print the counts for each treatment
# for treatment, count in treatment_counts.items():
#     print(f"Treatment {treatment + 1}: {count} times")

## Modelo de asignacion que itera por cluster

In [ ]:
# -------------------------------
# Procesamiento y preprocesamiento de datos
# -------------------------------

# Definir la carpeta base y el mapeo de tratamientos
base_folder = "cluster_data_20241106_020021/"  # Carpeta donde se encuentran los archivos de datos
tratamiento_map = {  # Mapeo de los tratamientos específicos a identificadores numéricos
    "Ejecutivo=0, Correos=0": 1, "Ejecutivo=0, Correos=1": 2,
    "Ejecutivo=0, Correos=2": 3, "Ejecutivo=0, Correos=3": 4,
    "Ejecutivo=0, Correos=4": 5, "Ejecutivo=1, Correos=0": 6,
    "Ejecutivo=1, Correos=1": 7, "Ejecutivo=1, Correos=2": 8
}

# Definir las rutas de los archivos
probabilities_file = os.path.join(base_folder, "probabilities_treatment.csv")
rut_info_file = os.path.join(base_folder, "rut_info.csv")
cluster_info_file = os.path.join(base_folder, "cluster_info.csv")

# Parámetros
costosms = 100  # Costo de cada mensaje SMS
capacidad_ejecutivos = 205000  # Capacidad máxima en términos de tiempo de los ejecutivos

# Paso 1: Cargar y mapear 'tratamiento_id' en los datos de probabilidades
print("Loading and processing probabilities data...")
df_probabilities = df_probabilities_treatment
df_probabilities['tratamiento_id'] = df_probabilities['Tratamiento'].map(tratamiento_map)

# Paso 2: Crear lista de tratamientos y combinar con rut_info
print("Merging probabilities with rut_info...")
df_probabilities['tratamientos'] = df_probabilities[['probabilidad_simular', 'tratamiento_id']].values.tolist()
grouped_prob = df_probabilities.groupby('categoria_clusterizacion_numerica')['tratamientos'].apply(list).reset_index()

df_rut_info = df_rut_info
df_rut_info = df_rut_info.merge(grouped_prob, on='categoria_clusterizacion_numerica', how='left')

# Paso 3: Combinar rut_info con cluster_info
print("Merging rut_info with cluster_info...")
df_cluster_info = df_cluster_info
df_rut_info = df_rut_info.merge(df_cluster_info, on='categoria_clusterizacion_numerica', how='left')

# Paso 3.5: Agrupar información por cluster en 'rut_info'
# Agrupar por 'categoria_clusterizacion_numerica' y agregar según lo especificado
df_grouped = df_rut_info.groupby('categoria_clusterizacion_numerica').agg({
    'Probabilidad_No_Pago': 'mean',  # Promedio de probabilidad de no pago
    'tratamientos': lambda x: list(x),  # Lista de opciones de tratamiento únicas en cada cluster
    'Monto_Simulado_mean': 'mean',
    'Plazo_Simulado_mean': 'mean',
    'probabilidad_aceptacion_optima': 'mean',
    'tasa_optima': 'mean',
    'rut': 'count'  # Conteo del número de clientes ('rut') en cada cluster
}).rename(columns={'rut': 'n_clientes'}).reset_index()

# Paso 4: Calcular 'RC' (Revenue calculado)
print("Calculating RC...")
df_grouped['tasa_optima'] /= 100  # Convertir tasa óptima a decimal
df_grouped['RC'] = (
    (df_grouped['Plazo_Simulado_mean'] * df_grouped['Monto_Simulado_mean'] * df_grouped['tasa_optima'] *
     ((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean'])) /
    (((1 + df_grouped['tasa_optima']) ** df_grouped['Plazo_Simulado_mean']) - 1)
) - df_grouped['Monto_Simulado_mean']

# -------------------------------
# Preparación de datos para optimización
# -------------------------------

# Convertir 'tratamientos' a un arreglo de numpy para mejorar la indexación
# Desarrollar y preparar 'tratamientos' para indexación adecuada
profits = np.array([
    [
        row['n_clientes'] * (row['RC'] * (1 - row['Probabilidad_No_Pago']) * row['probabilidad_aceptacion_optima'] * row['tratamientos'][0][t][0]) - 
        (row['tratamientos'][0][t][1] * costosms)
        for t in range(8)
    ]
    for _, row in df_grouped.iterrows()
])

# Inicializar el modelo de optimización
model = Model("Maximizar_Ganancias")
model.ModelSense = GRB.MAXIMIZE

# Crear variables de decisión y definir el objetivo
n_clients, n_treatments = profits.shape
variables = {}

for i in range(n_clients):
    variables[i] = {}
    for t in range(n_treatments):
        if profits[i, t] > 0:
            variables[i][t] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{t}")

model.setObjective(
    quicksum(variables[i][t] * profits[i, t] for i in variables for t in variables[i])
)

# Restricción: Cada cliente recibe exactamente un tratamiento
for i in variables:
    model.addConstr(quicksum(variables[i].values()) == 1, name=f"OneTreatmentPerClient_{i}")

# Restricción de capacidad para ejecutivos
model.addConstr(
    quicksum(variables[i][t] * df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i] if t in [5, 6, 7]) <= capacidad_ejecutivos,
    name="CapacityConstraint"
)

# Consistencia de cluster: los clientes dentro del mismo cluster deben recibir el mismo tratamiento
clusters = df_grouped.groupby("categoria_clusterizacion_numerica").indices
for cluster_id, indices_cluster in clusters.items():
    indices_list = list(indices_cluster)
    leader_index = indices_list[0]
    for t in variables[leader_index]:
        leader_var = variables[leader_index][t]
        for i in indices_list[1:]:
            if t in variables[i]:
                model.addConstr(variables[i][t] == leader_var, name=f"ClusterConsistency_{cluster_id}_{t}")

# Optimizar el modelo
model.optimize()

# Verificar si la optimización fue exitosa
if model.Status == GRB.OPTIMAL:
    # -------------------------------
    # Extracción y visualización de resultados
    # -------------------------------

    print("Extracting results...")

    # Asignar tratamientos por cluster basado en los resultados de la optimización
    resultados_por_cluster = {}
    for cluster_id, indices_cluster in clusters.items():
        leader_index = list(indices_cluster)[0]
        for t in variables[leader_index]:
            if variables[leader_index][t].X > 0.5:
                resultados_por_cluster[cluster_id] = t + 1
                break

    # Calcular las ganancias totales
    ganancias_totales = model.ObjVal

    # Mostrar resultados
    print("\nTratamientos asignados por cluster:")
    for cluster_id, tratamiento in resultados_por_cluster.items():
        print(f"Cluster {cluster_id}: Tratamiento {tratamiento + 1}")

    print(f"\nGanancias totales: {ganancias_totales:.2f}")

    # Calcular el número de ejecutivos usados y restantes
    executives_used = sum(
        df_grouped.loc[i, 'n_clientes'] for i in variables for t in variables[i]
        if t in [5, 6, 7] and variables[i][t].X > 0.5
    )
    executives_remaining = capacidad_ejecutivos - executives_used

    # Mostrar resumen de uso de ejecutivos
    print(f"\nExecutives used: {executives_used}")
    print(f"Executives remaining: {executives_remaining}")
else:
    print("Optimization did not reach an optimal solution.")
print("Optimization complete.")


Loading and processing probabilities data...
Merging probabilities with rut_info...
Merging rut_info with cluster_info...
Calculating RC...


NameError: name 'Model' is not defined

In [ ]:
# Contar cuántas veces se asigna cada tratamiento en los resultados por cluster
# Se utiliza un diccionario 'Counter' para contar las ocurrencias de cada tratamiento asignado en 'resultados_por_cluster'
for cluster_id, tratamiento in resultados_por_cluster.items():
    treatment_counts = Counter(resultados_por_cluster.values())

# Imprimir el conteo de asignaciones para cada tratamiento
# Se recorre 'treatment_counts' para mostrar cuántas veces se asignó cada tratamiento.
# 'treatment + 1' se utiliza para mostrar el número de tratamiento en base 1, haciendo el resultado más legible.
for treatment, count in treatment_counts.items():
    print(f"Treatment {treatment + 1}: {count} times")


Treatment 8: 38 times
Treatment 1: 2 times
Treatment 7: 10 times
Treatment 6: 2 times
Treatment 4: 10 times
Treatment 5: 41 times
Treatment 3: 5 times


In [ ]:
# Convertir el diccionario 'resultados_por_cluster' a un DataFrame
# El diccionario 'resultados_por_cluster' contiene el ID del cluster y el tratamiento asignado a cada uno.
# Se convierte a un DataFrame donde la primera columna es 'cluster' y la segunda 'assigned_treatment'.
df_resultados_por_cluster = pd.DataFrame(list(resultados_por_cluster.items()), columns=["cluster", "assigned_treatment"])

# Mostrar el DataFrame resultante al usuario
df_resultados_por_cluster


,cluster,assigned_treatment
0,0,8
1,1,1
2,2,7
3,3,6
4,4,4
...,...,...
103,103,5
104,104,5
105,105,7
106,106,5


In [ ]:
# Primer merge con 'df_rut_info'
# Realizar una unión ('merge') entre 'df_resultados_por_cluster' y 'df_rut_info' usando 'cluster' en el primer DataFrame
# y 'categoria_clusterizacion_numerica' en el segundo como claves de unión.
# Esta unión permite agregar información de clientes a cada cluster con su tratamiento asignado.
df_assigned = pd.merge(df_resultados_por_cluster, df_rut_info, left_on='cluster', right_on='categoria_clusterizacion_numerica', how='left')

# Segundo merge con 'df_grouped' para agregar la columna 'RC'
# Realizar una unión entre 'df_assigned' y 'df_grouped' para incorporar la columna 'RC' (Revenue Calculado)
# Usamos 'categoria_clusterizacion_numerica' como clave de unión para añadir la información de revenue calculado a cada cluster.
df_assigned = pd.merge(df_assigned, df_grouped[['categoria_clusterizacion_numerica', 'RC']], on='categoria_clusterizacion_numerica', how='left')


In [ ]:
# Calcular la probabilidad de simulación para el tratamiento asignado en cada fila de 'df_assigned'
# Se usa 'apply' con una función lambda para extraer la probabilidad de simulación correspondiente al tratamiento asignado.
# 'row['tratamientos']' es una lista de opciones de tratamiento y 'row['assigned_treatment'] - 1' indica la posición del tratamiento.
# Primero, se verifica que 'tratamientos' sea una lista y que el índice calculado esté dentro del rango.
# Si estas condiciones se cumplen, se extrae la probabilidad de simulación; en caso contrario, se asigna None.
df_assigned['probabilidad_de_simular'] = df_assigned.apply(
    lambda row: row['tratamientos'][row['assigned_treatment'] - 1][0] 
                if isinstance(row['tratamientos'], list) and (0 <= row['assigned_treatment'] - 1 < len(row['tratamientos'])) 
                else None,
    axis=1
)


In [ ]:
import os
from datetime import datetime

# Crear una carpeta con un nombre basado en la fecha y hora actual
# 'strftime' genera un timestamp en el formato "YYYYMMDD_HHMMSS" para asegurar nombres de carpeta únicos.
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
folder_name = f"assigned_treatments/assignation_{timestamp}"
os.makedirs(folder_name, exist_ok=True)  # Crear la carpeta; 'exist_ok=True' evita errores si la carpeta ya existe.

# Definir la ruta del archivo CSV dentro de la nueva carpeta
output_path = os.path.join(folder_name, 'assigned_treatments.csv')

# Guardar el DataFrame 'df_assigned' con las columnas seleccionadas en un archivo CSV
# Se incluyen las columnas clave: 'rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment',
# 'probabilidad_de_simular', 'tasa_optima' y 'probabilidad_aceptacion_optima'.
df_assigned[['rut', 'cluster', 'Probabilidad_No_Pago', 'RC', 'assigned_treatment', 'probabilidad_de_simular', 'tasa_optima', 'probabilidad_aceptacion_optima']].to_csv(output_path, index=False)

# Imprimir mensaje de confirmación con la ubicación del archivo CSV guardado
print(f"CSV file saved in folder: {output_path}")


CSV file saved in folder: assigned_treatments/assignation_20241106_222007\assigned_treatments.csv
